<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/ML_for_stocks_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime

# Load the dataset
file_path = "/content/sample_data/SMCI.csv"
data = pd.read_csv(file_path)

# Convert 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Sort the data by date
data.sort_values('Date', inplace=True)

# Set 'Date' as the index
data.set_index('Date', inplace=True)

# Ensure all 2024 data is included in the training set
train_end_date = datetime(2024, 7, 4)
train_data = data[data.index <= train_end_date]

# Feature Engineering: Including previous days' close prices as features
for lag in range(1, 8):
    train_data[f'Close_lag_{lag}'] = train_data['Close'].shift(lag)

# Drop rows with NaN values created by the lag features
train_data.dropna(inplace=True)

# Define the feature set and target
X = train_data.drop(columns=['Close'])
y = train_data['Close']

# Standardize the feature set
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


<ipython-input-1-991d554ddd24>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[f'Close_lag_{lag}'] = train_data['Close'].shift(lag)
<ipython-input-1-991d554ddd24>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[f'Close_lag_{lag}'] = train_data['Close'].shift(lag)
<ipython-input-1-991d554ddd24>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score

# Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_cv_score = cross_val_score(lr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_cv_score = cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Support Vector Machine
svm = SVR(kernel='rbf')
svm.fit(X_train, y_train)
svm_cv_score = cross_val_score(svm, X_train, y_train, cv=5, scoring='neg_mean_squared_error')


In [5]:
# Prepare the last data point for prediction (using the latest available data)
# Use the last available date in the index instead of train_end_date
last_data_point = data.iloc[-1]

# Create lagged features for the last data point
for lag in range(1, 8):
    last_data_point[f'Close_lag_{lag}'] = data['Close'].shift(lag).iloc[-1]

last_data_point_scaled = scaler.transform([last_data_point.drop('Close').values])

# ... rest of your code ...

# ... rest of your code ...
# Function to predict future values
def predict_future_values(model, current_data, days):
    predictions = []
    for _ in range(days):
        next_pred = model.predict(current_data)
        predictions.append(next_pred[0])

        # Prepare the next data point (shift data and include new prediction)
        current_data = np.roll(current_data, -1)
        current_data[0, -1] = next_pred
    return predictions

# Predict the next 7 days
days_to_predict = 7
predictions_lr = predict_future_values(lr, last_data_point_scaled, days_to_predict)
predictions_rf = predict_future_values(rf, last_data_point_scaled, days_to_predict)
predictions_svm = predict_future_values(svm, last_data_point_scaled, days_to_predict)

# Generate the dates for the predictions
last_date = train_end_date
predicted_dates = [last_date + pd.Timedelta(days=i) for i in range(1, days_to_predict + 1)]


<ipython-input-5-fa717ff99411>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_data_point[f'Close_lag_{lag}'] = data['Close'].shift(lag).iloc[-1]
<ipython-input-5-fa717ff99411>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_data_point[f'Close_lag_{lag}'] = data['Close'].shift(lag).iloc[-1]
<ipython-input-5-fa717ff99411>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_data_point[f'Close_lag_{lag}'] = data['Close'].shi

In [6]:
# Create a DataFrame to hold the predictions
predictions_df = pd.DataFrame({
    'Date': predicted_dates,
    'Linear Regression': predictions_lr,
    'Random Forest': predictions_rf,
    'SVM': predictions_svm
})

print(predictions_df)


        Date  Linear Regression  Random Forest         SVM
0 2024-07-05         846.580017     846.308396  256.942333
1 2024-07-06         387.509422     723.459399  197.137811
2 2024-07-07         849.948590     796.744093  197.137811
3 2024-07-08         842.960560     796.965792  197.137811
4 2024-07-09         820.594904     778.035385  197.137811
5 2024-07-10         830.435953     830.898182  197.137811
6 2024-07-11         905.936493     906.296287  197.137811


In [7]:
import plotly.graph_objs as go
import plotly.express as px

# Create traces for each model's predictions
trace_lr = go.Scatter(x=predictions_df['Date'], y=predictions_df['Linear Regression'], mode='lines+markers', name='Linear Regression')
trace_rf = go.Scatter(x=predictions_df['Date'], y=predictions_df['Random Forest'], mode='lines+markers', name='Random Forest')
trace_svm = go.Scatter(x=predictions_df['Date'], y=predictions_df['SVM'], mode='lines+markers', name='SVM')

# Create a layout
layout = go.Layout(
    title='Stock Price Predictions for the Next 7 Days',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Predicted Stock Price'),
    hovermode='closest'
)

# Create a figure
fig = go.Figure(data=[trace_lr, trace_rf, trace_svm], layout=layout)

# Show the figure
fig.show()
